In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import heapq
import plotly.graph_objects as go
import webbrowser
from itertools import combinations
from scipy.spatial.distance import euclidean
import random

In [ ]:
df = pd.read_csv("tum_sentetik2.csv")
df['Kategori'] = df['Mağaza'].apply(lambda x: x.split('_')[0])

available_products = df['Ürün'].unique()
product_to_category = df.groupby('Ürün')['Kategori'].first().to_dict()

def get_shopping_list(available_products, product_to_category):
    print("\n=== Alışveriş Listesi Oluşturma ===")
    
    shopping_list = []
    print("\nAlışveriş listesine eklemek istediğiniz ürünleri girin.")
    print("Bitirmek için 'bitir' yazın.")
    
    while True:
        product = input("Ürün adı: ").strip()
        if product.lower() == 'bitir':
            break
        if product in available_products:
            shopping_list.append(product)
            print(f"{product} listeye eklendi.")
        else:
            print(f"Hata: '{product}' veri setinde bulunmuyor. Lütfen geçerli bir ürün girin.")
    
    if not shopping_list:
        raise ValueError("Alışveriş listesi boş olamaz!")
    
    print("\nOluşturulan Alışveriş Listesi:", shopping_list)
    return shopping_list

# Alışveriş listesini kullanıcıdan al
shopping_list = get_shopping_list(available_products, product_to_category)

# Ürün kategorilerini oluştur
item_categories = {item: product_to_category.get(item, "Bilinmeyen") for item in shopping_list}

# Mağaza kategorileri için renkler
kategori_renkleri = {
    "Gida": "orange", "Kozmetik": "purple", "Giysi": "blue", "Elektronik": "red", "Ev": "green"
}

# Ürünleri içeren mağazaları bul
filtered_stores = df[df['Ürün'].isin(shopping_list)][['Mağaza', 'X_Koordinat', 'Y_Koordinat', 'Kategori']].drop_duplicates()
valid_stores = set(filtered_stores['Mağaza'])
store_products = df[df['Ürün'].isin(shopping_list) & df['Mağaza'].isin(valid_stores)].groupby('Mağaza')['Ürün'].apply(set).to_dict()
product_prices = df[df['Ürün'].isin(shopping_list) & df['Mağaza'].isin(valid_stores)].set_index(['Mağaza', 'Ürün'])['Fiyat'].to_dict()

# Ürün-Mağaza eşleştirmesi
item_to_stores = {item: set() for item in shopping_list}
for store in filtered_stores['Mağaza']:
    store_category = store.split('_')[0]
    for item in store_products.get(store, set()):
        if item in shopping_list and item_categories[item] == store_category:
            item_to_stores[item].add(store)

In [ ]:
# Graf oluştur
G_astar = nx.Graph()
for _, row in filtered_stores.iterrows():
    G_astar.add_node(row['Mağaza'], pos=(row['X_Koordinat'], row['Y_Koordinat']), kategori=row['Kategori'])

for (m1, x1, y1), (m2, x2, y2) in combinations(filtered_stores[['Mağaza', 'X_Koordinat', 'Y_Koordinat']].values, 2):
    distance = euclidean((x1, y1), (x2, y2))
    travel_time = distance / 60  # Saat cinsinden
    G_astar.add_edge(m1, m2, weight=travel_time)

# Mağaza konumlarını al
pos = nx.get_node_attributes(G_astar, 'pos')

start_node = random.choice(list(G_astar.nodes()))
print(f"\nRastgele seçilen başlangıç mağazası: {start_node}")

### A* Algo Functions

In [ ]:
def heuristic(node, goal, pos):
    x1, y1 = pos[node]
    x2, y2 = pos[goal]
    return euclidean((x1, y1), (x2, y2)) / 60  # Saat cinsinden mesafe

# A* Algoritması (Ulaşım maliyeti saat * 100 ile entegre)
def astar_optimized_shopping(graph, start, pos, product_prices, store_products, shopping_list):
    priority_queue = []
    heapq.heappush(priority_queue, (0, start, [start], 0, set(), {}))  
    # (toplam maliyet, düğüm, yol, toplam fiyat, alınan ürünler, hangi mağazadan hangi ürünler alındı)

    visited = set()

    while priority_queue:
        total_cost, current, path, total_price, collected_items, items_source = heapq.heappop(priority_queue)

        # Tüm ürünleri aldıysak bu yol en iyisi olabilir
        if collected_items == set(shopping_list):
            return path, total_cost, total_price, items_source

        if (current, frozenset(collected_items)) in visited:
            continue

        visited.add((current, frozenset(collected_items)))

        for neighbor in graph.neighbors(current):
            travel_time = graph[current][neighbor]['weight']  # Yol süresi (saat cinsinden)
            travel_cost = travel_time * 100  # Ulaşım maliyeti (saat * 100 TL)

            # Bu mağazada hangi ürünler var?
            available_products = store_products.get(neighbor, set())

            # Hangi ürünleri buradan almak daha avantajlı?
            best_products = set()
            for product in set(shopping_list) - collected_items:
                if product in available_products:
                    if product in items_source:
                        old_store, old_price = items_source[product]
                        new_price = product_prices.get((neighbor, product), float('inf'))
                        if new_price < old_price:
                            best_products.add(product)
                    else:
                        best_products.add(product)

            # Bu mağazadan alınacak ürünlerin fiyatını hesapla
            new_item_price = sum(
                product_prices.get((neighbor, product), float('inf')) for product in best_products
            )

            # Yeni toplam maliyet (ürün fiyatı + ulaşım maliyeti)
            new_total_price = total_price + new_item_price
            new_total_cost = total_cost + travel_cost + new_item_price
            heuristic_cost = heuristic(neighbor, start, pos) * 100  # Heuristic maliyeti de saat * 100 ile uyumlu

            # Yeni maliyet (G + H)
            cost_with_heuristic = new_total_cost + heuristic_cost

            # Yeni alınan ürünleri güncelle
            new_collected_items = collected_items.union(best_products)
            new_items_source = items_source.copy()
            for product in best_products:
                new_items_source[product] = (neighbor, product_prices.get((neighbor, product), float('inf')))

            heapq.heappush(priority_queue, (cost_with_heuristic, neighbor, path + [neighbor], new_total_price, new_collected_items, new_items_source))

    return None, float('inf'), float('inf'), {}  # Yol bulunamazsa

In [2]:
# A* Algoritmasını çalıştır
optimal_path, total_cost, total_price, items_source = astar_optimized_shopping(G_astar, start_node, pos, product_prices, store_products, shopping_list)

from IPython.display import display
display(optimal_path)

print("+++++++++++++++++++++++")

# Sonuçları yazdır
print("\n=== A* Algoritma Sonuçları ===")
print("Optimal Rota:")
for i, store in enumerate(optimal_path, 1):
    print(f"  {i}. {store}")
print(f"\nToplam Maliyet (Ulaşım + Ürün): {total_cost:.2f} TL")
print(f"Toplam Ürün Fiyatı: {total_price:.2f} TL")
print("\nÜrün Kaynakları:")
for item in shopping_list:
    if item in items_source:
        store, price = items_source[item]
        print(f"  {item}: {store} (Fiyat: {price:.2f} TL)")
    else:
        print(f"  {item}: Bulunamadı")
missing_items = set(shopping_list) - set(items_source.keys())
if not missing_items:
    print("\nTüm ürünler başarıyla alındı!")
else:
    print("\nEksik ürünler:", missing_items)

# Görselleştirme 
node_x = [pos[node][0] for node in G_astar.nodes()]
node_y = [pos[node][1] for node in G_astar.nodes()]
node_color = ["black" if node in optimal_path else kategori_renkleri.get(G_astar.nodes[node]['kategori'], 'gray') for node in G_astar.nodes()]
node_text = [node if node in optimal_path else "" for node in G_astar.nodes()]

edge_x = []
edge_y = []
for i in range(len(optimal_path) - 1):
    x0, y0 = pos[optimal_path[i]]
    x1, y1 = pos[optimal_path[i + 1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

fig = go.Figure()
fig.add_trace(go.Scatter(x=edge_x, y=edge_y, line=dict(width=2, color='gray'), mode='lines', name="Rota"))
fig.add_trace(go.Scatter(
    x=node_x, y=node_y, mode='markers+text',
    marker=dict(size=10, color=node_color),
    text=node_text, textposition="top center", hoverinfo="text"
))
fig.update_layout(
    title="Optimal Alışveriş Rotası ve Ürün Seçimi (A* Algoritması)",
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)
fig.write_html("astar.html")
webbrowser.open("astar.html")


=== Alışveriş Listesi Oluşturma ===

Alışveriş listesine eklemek istediğiniz ürünleri girin.
Bitirmek için 'bitir' yazın.


Avakado listeye eklendi.
Labne listeye eklendi.
Sucuk listeye eklendi.
Laptop listeye eklendi.
Ruj listeye eklendi.
Makas listeye eklendi.
Patates listeye eklendi.
Bulaşık Deterjanı listeye eklendi.
Mont listeye eklendi.
Tişört listeye eklendi.

Oluşturulan Alışveriş Listesi: ['Avakado', 'Labne', 'Sucuk', 'Laptop', 'Ruj', 'Makas', 'Patates', 'Bulaşık Deterjanı', 'Mont', 'Tişört']

Rastgele seçilen başlangıç mağazası: Elektronik_Magaza_20


['Elektronik_Magaza_20',
 'Giysi_Magaza_33',
 'Ev_Magaza_3',
 'Gida_Magaza_0',
 'Kozmetik_Magaza_1',
 'Ev_Magaza_39',
 'Gida_Magaza_42',
 'Giysi_Magaza_44',
 'Elektronik_Magaza_4']

+++++++++++++++++++++++

=== A* Algoritma Sonuçları ===
Optimal Rota:
  1. Elektronik_Magaza_20
  2. Giysi_Magaza_33
  3. Ev_Magaza_3
  4. Gida_Magaza_0
  5. Kozmetik_Magaza_1
  6. Ev_Magaza_39
  7. Gida_Magaza_42
  8. Giysi_Magaza_44
  9. Elektronik_Magaza_4

Toplam Maliyet (Ulaşım + Ürün): 2809.45 TL
Toplam Ürün Fiyatı: 1643.38 TL

Ürün Kaynakları:
  Avakado: Gida_Magaza_0 (Fiyat: 16.01 TL)
  Labne: Gida_Magaza_42 (Fiyat: 90.20 TL)
  Sucuk: Gida_Magaza_42 (Fiyat: 70.62 TL)
  Laptop: Elektronik_Magaza_4 (Fiyat: 520.15 TL)
  Ruj: Kozmetik_Magaza_1 (Fiyat: 71.81 TL)
  Makas: Ev_Magaza_3 (Fiyat: 112.88 TL)
  Patates: Gida_Magaza_42 (Fiyat: 37.71 TL)
  Bulaşık Deterjanı: Ev_Magaza_39 (Fiyat: 244.18 TL)
  Mont: Giysi_Magaza_33 (Fiyat: 225.64 TL)
  Tişört: Giysi_Magaza_44 (Fiyat: 254.18 TL)

Tüm ürünler başarıyla alındı!


True